# Бибилиотеки

In [3]:
import pandas as pd
import numpy as np

In [4]:
import datetime

In [23]:
!pip install PyDrive

In [24]:
# Установка из драйва
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [25]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

urls= ['1kX6NAsr50aXSnuYyO9eCd5YiaPwTQmJC','1m5QO_nGsJiDNn4Dd8vFefIg6GiZUB23m',
       '1S7VC63UML5K1TLYkpAjQdkYnosxEWEJc','16oOkTgnlYYPFKkKrfivsm8XTeVbaY0ba',
       '1bHJ-q2tLK2hB0Hz4wSF8miqy1-daZEKe','1E80vehSzvmV3vmunOc2CY9twMa4FcTcb',
       '1EfLkr0jFKNFq54B4bTCn1a5aWr03eDOV','1B2RJmc2naDGc3ZCHMWsHjv74V5yw4BV8',
       '10kXYfu48klaNaMCtozYrGNJkZiHKLq5F','1AZ2lp_S7mestktrxL8kkWC_BaXQf_sId',
       '16sT26KyRXvr46KGhLyesOhl_UnX-yxmR','1wIQuZfPEqpWcXn-crxuoOEF5sT0sHKbT']

for c in urls:
    file_id = c
    downloaded = drive.CreateFile({'id':file_id})
    downloaded.FetchMetadata(fetch_all=True)
    downloaded.GetContentFile(downloaded.metadata['title'])

# Подготовка данных

## Книги

Спецификация

*   Выдача 1 - delivBook1
*   Выдача 2 - delivBook2
*   Читатели - readers
*   Каталог - catalog
*   Экземпляры 1 - bone1
*   Экземпляры 2 - bone2


In [5]:
delivBook1 = pd.read_excel('/content/Выдача_1.xlsx').drop(columns=['Состояния', 'ИД выдачи',
                                                                   'Unnamed: 7', 'Unnamed: 9',
                                                                   'Инвентарный номер'])
delivBook2 = pd.read_excel('/content/Выдача_2.xlsx').drop(columns=['Состояния', 'ИД выдачи',
                                                                   'Unnamed: 7', 'Unnamed: 9',
                                                                   'Инвентарный номер'])
readers = pd.read_excel('/content/Читатели.xlsx').drop(columns=[])
bone1 = pd.read_excel('/content/Экземпляры.xlsx').drop(columns=['Идентификатор сиглы','Инвентарный номер','Идентификатор экземпляра'])
bone2 = pd.read_excel('/content/Экземпляры_2.xlsx').drop(columns=['Идентификатор сиглы','Инвентарный номер','Идентификатор экземпляра'])

In [157]:
catalog = pd.read_excel('/content/Каталог.xlsx').drop(columns=['p084a'])
catalog['genre'] = catalog.p650a.str.replace('. ',';').str.split(';',expand=True)[0].copy()

Сформируем данные по Бибилиотекам

In [ ]:
readers['Дата рождения'] = readers['Дата рождения'].apply(lambda x: x[:10] if isinstance(x, str) else x)
readers['Дата рождения'] = readers['Дата рождения'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d') if isinstance(x, str) else x)
readers['Возраст'] = 0
for i in range(len(readers)):
    readers['Возраст'].iloc[i] = datetime.datetime.now().year - readers['Дата рождения'].iloc[i].year

In [ ]:
delivBook1 = pd.merge(delivBook1, readers[['ID читателя','Возраст']], how='left', left_on='ИД читателя', right_on='ID читателя')
delivBook1 = delivBook1.drop(columns=['ID читателя'])
delivBook1['Дата выдачи'] = delivBook1['Дата выдачи'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if isinstance(x, str) else x)
delivBook1['Дата сдачи (предполагаемая)'] = delivBook1['Дата сдачи (предполагаемая)'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if isinstance(x, str) else x)
delivBook1['Период'] = 0
for i in range(len(delivBook1)):
    k = delivBook1['Дата сдачи (предполагаемая)'].iloc[i] - delivBook1['Дата выдачи'].iloc[i]
    delivBook1['Период'].iloc[i] = k.days

In [ ]:
delivBook2 = pd.merge(delivBook2, readers[['ID читателя','Возраст']], how='left', left_on='ИД читателя', right_on='ID читателя')
delivBook2 = delivBook2.drop(columns=['ID читателя'])
delivBook2['Дата выдачи'] = delivBook2['Дата выдачи'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if isinstance(x, str) else x)
delivBook2['Дата сдачи (предполагаемая)'] = delivBook2['Дата сдачи (предполагаемая)'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') if isinstance(x, str) else x)
delivBook2['Период'] = 0
for i in range(len(delivBook2)):
    k = delivBook2['Дата сдачи (предполагаемая)'].iloc[i] - delivBook2['Дата выдачи'].iloc[i]
    delivBook2['Период'].iloc[i] = k.days

In [ ]:
delivBook = delivBook1.append(delivBook2, ignore_index=True)

In [ ]:
bone = bone1.append(bone2, ignore_index=True)

In [ ]:
pred_param = catalog[['doc_id','genre']].copy()
pred_param = pd.merge(bone, pred_param, how='left', left_on='ИД Каталожной записи', right_on='doc_id')
pred_param = pred_param.dropna()
pred_param = pred_param.drop(columns=['doc_id'])

In [ ]:
book_user = pd.merge(delivBook, pred_param, how='left', left_on='Штрих-код', right_on='Штрих-код').dropna()
book_user['Возраст'] = book_user['Возраст'].astype(int)
book_user['ИД Каталожной записи'] = book_user['ИД Каталожной записи'].astype(int)
book_user.to_csv('book_user.csv', index=False)

## Мероприятия

Сформируем данные по Мероприятиям

In [202]:
event = pd.read_excel('/content/Мероприятия.xlsx')

In [203]:
event[event['Направленность мероприятия'].isna()] = 'без категории'

In [204]:
event['Код категории мероприятия'] = event['Направленность мероприятия'].str.split(',',expand=True)[0].copy().astype('category').cat.codes

In [205]:
event_sort = event[['id','Статус','Направленность мероприятия',
                    'Дата начала мероприятия','Дата окончания мероприятия',
                    'Место проведения','Доступность мероприятия для лиц с ОВЗ',
                    'Возрастная категория','Код категории мероприятия']].copy()

In [206]:
event_sort = event_sort.loc[(event_sort['Статус'] == 'Утверждено') |
                            (event_sort['Статус'] == 'Запланировано') |
                            (event_sort['Статус'] == 'Опубликовано')].copy()

In [207]:
event_sort['Дата начала мероприятия'] = event_sort['Дата начала мероприятия'].apply(lambda x: datetime.datetime.strptime(x, '%d.%m.%Y %H:%M') if isinstance(x, str)  else x)

Оставим для выпуска данные только за 45 дней вперед

In [208]:
filter_event = []
for i in range(len(event_sort)):
    if (event_sort['Дата начала мероприятия'].iloc[i] - datetime.datetime.now()) < datetime.timedelta(45, 0, 0):
        filter_event.append(i)

In [209]:
event_sort = event_sort.iloc[filter_event].copy()

In [210]:
event_sort = event_sort[event_sort['Место проведения'] != 'Литовский бульв. 7, Арт-клуб: 2 этаж, помещение Iа, ком. № 3']

In [211]:
event_sort = event_sort[event_sort['Место проведения'] != '127482, город Москва, город Зеленоград, корпус 401, Центральная детская библиотека №251']

In [212]:
event_sort = event_sort[event_sort['Место проведения'] != 'Двадцати Шести Бакинских Комиссаров ул. 12, корпус 2']

In [217]:
event_sort = event_sort[event_sort['Место проведения'] != 'Литовский бульв. 7, Малый зал: 2 этаж, помещение II, ком. № 1']

In [231]:
!pip install yandex-geocoder

In [232]:
from decimal import Decimal
from yandex_geocoder import Client
#Key API (Вставить ключ API)
client = Client("9e1317de-547c-44c8-ab1b-b6b5a088ba91")

In [233]:
def _coordinate(df):
    df['latitude'] = 0
    df['longitude'] = 0
    for i in range(len(df)):
        df['latitude'].iloc[i] = client.coordinates(df['Место проведения'].iloc[i])[0]
        df['longitude'].iloc[i] = client.coordinates(df['Место проведения'].iloc[i])[1]
    return df

In [214]:
event_sort[['from','to','unknow']] = event_sort['Возрастная категория'].str.replace('от ','').str.split(' до ',expand=True).copy()

In [215]:
event_sort = event_sort.drop(columns=['unknow'])

In [220]:
event_sort = _coordinate(event_sort)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

In [222]:
event_sort.to_csv('event.csv', index=False)

## Кружки

Сформируем данные по Кружкам

In [122]:
data = pd.read_csv('/content/Classificator_hachaton.csv', sep=';')
data1 = pd.read_csv('/content/MegaRelation_hackaton.csv', sep=';')
data2 = pd.read_csv('/content/Pupil_hackaton.csv', sep=';')
data3 = pd.read_csv('/content/org_hackaton.csv', sep=';')
data4 = pd.read_csv('/content/request_hackaton.csv', sep=';')
data5 = pd.read_csv('/content/services_hackaton.csv', sep=';')

In [226]:
print(data.head(2),data1.head(2),data2.head(2),data3.head(2),data4.head(2),data5.head(2))

   id_классификатора  id_родительского_классификатора Наименование
0            3002985                              NaN    Автошкола
1            3002986                        3002985.0     Автоклуб    id_зачисления  ...  Плановая_дата_окончания_занятий
0        1325954  ...               01.09.2021 0:00:00
1        2879918  ...               01.09.2021 0:00:00

[2 rows x 14 columns]    id_ученика  возраст      пол
0           1     12.0  Женский
1           2     12.0  Женский    id_организации  ...              Метро
0               1  ...                NaN
1               2  ...  Краснопресненская

[2 rows x 5 columns]    id_заявления  ... id_услуги
0        989578  ...     39178
1       3571456  ...     39178

[2 rows x 9 columns]    id_услуги Тип_финансирования  ...  длительность_обучения  единица_длительности
0      39178          бесплатно  ...                 7,0000                   лет
1      39181          бесплатно  ...                 7,0000                   лет

[2 ro

In [227]:
data.head(2)

,id_классификатора,id_родительского_классификатора,Наименование
0,3002985,NaN,Автошкола
1,3002986,3002985.0,Автоклуб


In [228]:
data1.head(2)

,id_зачисления,Статус,id_ученика,Дата_создания_записи,id_организации,id_заявления,id_услуги,дата_зачисления,дата_отчисления,причина_перевода,предыдущая_запись_зачисления,следующая_запись_зачисления,Плановая_дата_начала_занятий,Плановая_дата_окончания_занятий
0,1325954,3.0,25969.0,08.06.2015 14:47:19,30,950832.0,39178,08.06.2015 0:00:00,29.02.2016 19:28:12,NaN,NaN,3558114.0,01.09.2014 0:00:00,01.09.2021 0:00:00
1,2879918,3.0,158129.0,29.10.2015 18:35:21,30,989658.0,39178,01.09.2010 0:00:00,29.02.2016 19:23:59,NaN,NaN,3557885.0,01.09.2014 0:00:00,01.09.2021 0:00:00


In [229]:
data2.head(2)

,id_ученика,возраст,пол
0,1,12.0,Женский
1,2,12.0,Женский


In [230]:
data3.head(2)

,id_организации,полное_наименование,краткое_наименование,улица,Метро
0,1,Государственное бюджетное образовательное учре...,ГБОУ СПО г.Москвы «Московская средняя специаль...,NaN,NaN
1,2,Государственное бюджетное профессиональное обр...,"ГБПОУ г. Москвы ""Академия джаза""",улица Трёхгорный Вал,Краснопресненская


In [231]:
data4.head(2)

,id_заявления,статус_заявления,дата_создания,оценка_вступительных_испытаний,Испытания_пройдены,Зачисление_разрешено,Зачисление_не_разрешено,Причина_отказа_в_зачислении,id_услуги
0,989578,Ожидание прихода Заявителя на вступительные ис...,02.06.2014 17:25:37,NaN,0.0,0.0,0.0,NaN,39178
1,3571456,Архив,05.11.2015 18:51:59,5.0,1.0,0.0,NaN,NaN,39178


In [232]:
data5.head(2)

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
0,39178,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 12:50:32,"7,0000",лет
1,39181,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 13:04:41,"7,0000",лет


In [246]:
data5['длительность_обучения'] = data5['длительность_обучения'].str.split(',',expand=True)[0].copy()

In [249]:
data5['years'] = 0
data5['weeks'] = 0
data5['months'] = 0
data5['days'] = 0

for i in range(len(data5)):
    if data5['единица_длительности'].iloc[i] == 'лет':
        data5['years'] = data5['длительность_обучения'].iloc[i]
        data5['weeks'] = 0
        data5['months'] = 0
        data5['days'] = 0
    elif data5['единица_длительности'].iloc[i] == 'недель':
        data5['years'] = 0
        data5['weeks'] = data5['длительность_обучения'].iloc[i]
        data5['months'] = 0
        data5['days'] = 0
    elif data5['единица_длительности'].iloc[i] == 'месяцев':   
        data5['years'] = 0
        data5['weeks'] = 0
        data5['months'] = data5['длительность_обучения'].iloc[i]
        data5['days'] = 0 
    elif data5['единица_длительности'].iloc[i] == 'дней': 
        data5['years'] = 0
        data5['weeks'] = 0
        data5['months'] = 0
        data5['days'] = data5['длительность_обучения'].iloc[i]
    else:
        print(i)

6540
8106
9274
9275
16989
17085
17959
17960
21318
21380
22003
22925
24084
39183
39300
39549
39948
39949
40821


In [251]:
data5 = data5.drop(columns=['единица_длительности','длительность_обучения','Дата_создания'])

In [254]:
data5

,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,years,weeks,months,days
0,39178,бесплатно,3000001,30,Индивидуальное,Архитектура,5,0,0,0
1,39181,бесплатно,3000001,30,Индивидуальное,Архитектура,5,0,0,0
2,212072,бесплатно,3000001,30,Индивидуальное,Архитектура,5,0,0,0
3,212517,бесплатно,3000001,30,Индивидуальное,Архитектура,5,0,0,0
4,39913,бесплатно,3000003,45,Индивидуальное,Декоративно-прикладное искусство,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...
40967,660234,бесплатно,4507940,1800,Общее,"""Играю, рисую, пою""",5,0,0,0
40968,728015,бесплатно,4507940,29,Индивидуальное,Мир искусств,5,0,0,0
40969,987697,бесплатно,4507940,26,Индивидуальное,Фортепиано и комплексное музицирование,5,0,0,0
40970,1099087,бесплатно,4507940,29,Индивидуальное,РАС,5,0,0,0


In [266]:
data4 = data4.drop(columns=['Причина_отказа_в_зачислении','оценка_вступительных_испытаний',
                            'дата_создания','Испытания_пройдены','Зачисление_разрешено',
                            'Зачисление_не_разрешено'])

In [267]:
data4['зачислен'] = 0
data4['не_зачислен'] = 0
data4['сдал'] = 0

In [271]:
for i in range(len(data4)):
    if data4['статус_заявления'].iloc[i] == 'Ожидание прихода Заявителя на вступительные испытания':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 0
        data4['сдал'].iloc[i] = 0
    if data4['статус_заявления'].iloc[i] == 'Архив':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 1
        data4['сдал'].iloc[i] = 1
    if data4['статус_заявления'].iloc[i] == 'Отказ в зачислении':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 1
        data4['сдал'].iloc[i] = 0
    if data4['статус_заявления'].iloc[i] == 'Зачислен':
        data4['зачислен'].iloc[i] = 1
        data4['не_зачислен'].iloc[i] = 0
        data4['сдал'].iloc[i] = 1
    if data4['статус_заявления'].iloc[i] == 'Ожидание прихода Заявителя для заключения договора':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 0
        data4['сдал'].iloc[i] = 1
    if data4['статус_заявления'].iloc[i] == 'Отчислен':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 1
        data4['сдал'].iloc[i] = 0
    if data4['статус_заявления'].iloc[i] == 'Вступительные испытания пройдены':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 0
        data4['сдал'].iloc[i] = 1
    if data4['статус_заявления'].iloc[i] == 'Черновик':
        data4['зачислен'].iloc[i] = 0
        data4['не_зачислен'].iloc[i] = 0
        data4['сдал'].iloc[i] = 0  
    if data4['статус_заявления'].iloc[i] == 'Поданы документы':
        data4['зачислен'].iloc[i] = 1
        data4['не_зачислен'].iloc[i] = 0
        data4['сдал'].iloc[i] = 1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [273]:
data4['статус_код'] = data4['статус_заявления'].astype('category').cat.codes

In [274]:
freq_enc = data4["статус_заявления"].value_counts(normalize=True)
data4["статус_частота"] = data4["статус_заявления"].map(freq_enc)

In [282]:
data1 = data1[['id_заявления','id_услуги','id_ученика','Статус','Дата_создания_записи', 'дата_зачисления', 'дата_отчисления']].copy()

In [289]:
data1 = pd.merge(data5, data1, how='left', left_on=['id_услуги'], right_on=['id_услуги'])

In [297]:
data1 = data1[~data1['id_ученика'].isna()]

In [301]:
data1 = data1[~data1['id_заявления'].isna()]

In [302]:
data1['id_заявления'] = data1['id_заявления'].astype(int)
data1['id_ученика'] = data1['id_ученика'].astype(int)
data1['Статус'] = data1['Статус'].astype(int)

In [304]:
data1 = data1.drop(columns=['дата_зачисления','дата_отчисления'])

In [306]:
freq_enc = data1["Классификатор_услуги"].value_counts(normalize=True)
data1["Классификатор_услуги_частота"] = data1["Классификатор_услуги"].map(freq_enc)

In [307]:
data1 = data1.drop(columns=['Классификатор_услуги'])

In [308]:
data1['финансирования_код'] = data1['Тип_финансирования'].astype('category').cat.codes

In [309]:
data1['расписания_код'] = data1['Тип_расписания'].astype('category').cat.codes

In [313]:
data4 = pd.merge(data4, data1, how='left', left_on=['id_заявления','id_услуги'], right_on=['id_заявления','id_услуги']).dropna()

In [316]:
data2['пол'] = data2['пол'].astype('category').cat.codes

In [318]:
data4['id_ученика'] = data4['id_ученика'].astype(int)
data4['weeks'] = data4['weeks'].astype(int)
data4['months'] = data4['months'].astype(int)
data4['days'] = data4['days'].astype(int)
data4['id_организации'] = data4['id_организации'].astype(int)

In [321]:
data4 = pd.merge(data4, data2, how='left', left_on=['id_ученика'], right_on=['id_ученика']).dropna()

In [330]:
if np.isnan(data['id_родительского_классификатора'].iloc[1]):
    print('f')

In [331]:
data['id пользователя'] = 0
for i in range(len(data)):
    if np.isnan(data['id_родительского_классификатора'].iloc[i]):
        data['id пользователя'].iloc[i] = data['id_классификатора'].iloc[i]
    else:
        data['id пользователя'].iloc[i] = data['id_родительского_классификатора'].iloc[i]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [333]:
data = data.drop(columns=['id_родительского_классификатора'])

In [340]:
data_org = data4.drop(columns=['статус_заявления','id_заявления','id_услуги','Тип_финансирования','Тип_расписания','Дата_создания_записи']).copy()

In [341]:
data_org.to_csv('organiz.csv', index=False)

# Рекомендательный алгоритм

Исходные данные

In [1]:
!unzip /content/event_library.zip
!unzip /content/organiz.zip

Archive:  /content/event_library.zip
  inflating: book_user.csv           
  inflating: book_user_s.csv         
  inflating: event.csv               
Archive:  /content/organiz.zip
  inflating: organiz.csv             


In [ ]:
catalog = pd.read_excel('/content/Каталог.xlsx').drop(columns=['p084a'])
catalog['genre'] = catalog.p650a.str.replace('. ',';').str.split(';',expand=True)[0].copy()

In [268]:
data3 = pd.read_csv('/content/org_hackaton.csv', sep=';')

In [26]:

book_user = pd.read_csv('/content/book_user.csv')
event = pd.read_csv('/content/event.csv')
organiz = pd.read_csv('/content/organiz.csv')

In [27]:
book_user['pred_genre'] = book_user.genre.apply(lambda x: x[:1])

In [28]:
fronts = book_user.pred_genre.drop_duplicates().values

In [299]:
book_user.genre.apply(lambda x: x[:1]).drop_duplicates().sort_values().values

array(['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'К', 'Л', 'М', 'Н',
       'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Э', 'Ю',
       'Я'], dtype=object)

In [29]:
organiz['Наименование_услуги'] = organiz['Наименование_услуги'].str.replace('[(-+/\".&]','')

In [30]:
organiz['pred_genre'] = organiz['Наименование_услуги'].apply(lambda x: x[:1])

In [31]:
organiz.shape

(455817, 19)

In [32]:
oraniz_test = pd.DataFrame([])
for c in fronts:
    oraniz_test = oraniz_test.append(organiz[organiz.pred_genre == c])

In [33]:
oraniz_test.shape

(393398, 19)

In [34]:
book_user = book_user.sort_values(by=['pred_genre'])
oraniz_test = oraniz_test.sort_values(by=['pred_genre'])

In [35]:
book_user['classif'] = book_user.pred_genre.astype('category').cat.codes
oraniz_test['classif'] = oraniz_test.pred_genre.astype('category').cat.codes

In [36]:
book_user = book_user.drop(columns=['pred_genre'])
oraniz_test = oraniz_test.drop(columns=['pred_genre'])

Теперь категории имеют класс для связи, так же подредактировать этот класс и для мероприятий

In [37]:
freq_enc = book_user["Раздел знаний"].value_counts(normalize=True)
book_user["Раздел знаний"] = book_user["Раздел знаний"].map(freq_enc)

In [38]:
freq_enc = book_user["Состояние"].value_counts(normalize=True)
book_user["Состояние"] = book_user["Состояние"].map(freq_enc)

In [39]:
book_user = book_user.drop(columns=['Штрих-код','Дата выдачи','Дата сдачи (предполагаемая)'])

In [40]:
book_user['Возраст'] = book_user['Возраст'].astype(int)
book_user['ИД Каталожной записи'] = book_user['ИД Каталожной записи'].astype(int)

In [275]:
event.to_csv('final_event.csv', index=False)

Возрастные категории

In [51]:
def age_range(df,col):
    df['age_range'] = ''
    for i in range(len(df)):
        if df[col].iloc[i] < 15:
            df['age_range'].iloc[i] = 'до 14'
        elif df[col].iloc[i] > 14 and df[col].iloc[i] < 19:
            df['age_range'].iloc[i] = 'от 14 до 18'
        elif df[col].iloc[i] > 19 and df[col].iloc[i] < 25:
            df['age_range'].iloc[i] = 'от 18 до 24'
        elif df[col].iloc[i] > 25 and df[col].iloc[i] < 35:
            df['age_range'].iloc[i] = 'от 25 до 34'
        elif df[col].iloc[i] > 35 and df[col].iloc[i] < 55:
            df['age_range'].iloc[i] = 'от 35 до 55'
        else:
            df['age_range'].iloc[i] = 'от 55'
    return df

In [54]:
book_user = age_range(book_user,'Возраст')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [76]:
book_user = book_user.sort_values(by=['age_range'])

In [78]:
book_user['age_range_code'] = book_user.age_range.astype('category').cat.codes

In [294]:
book_user['genre'].drop_duplicates().sort_values()

280576        Автодорожны
118893          Автоматик
71085          Автоматика
49520     Административно
315169           Акустика
               ...       
247060              Этика
95713            Этнологи
269784                Юмо
61677       Юмористическа
75400              Ядерна
Name: genre, Length: 363, dtype: object

In [274]:
book_user.to_csv('final_book.csv', index=False)
oraniz_test.to_csv('final_org.csv', index=False)

Рекомендации по книгам

In [41]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import collections

In [289]:
train, test= train_test_split(book_user.drop(columns=['genre','age_range']).set_index(['ИД читателя']),stratify=book_user['classif'],
                              random_state = 4256,test_size=0.2)

In [290]:
train

,Состояние,Возраст,Период,ИД Каталожной записи,Раздел знаний,classif,age_range_code
ИД читателя,,,,,,,
55989,0.851124,66,23,8536,0.104621,20,5
424534,0.851124,33,0,416677,0.037986,8,3
8624,0.142162,73,31,5173,0.000344,20,5
14904,0.851124,42,26,3878,0.104621,20,4
47156,0.851124,39,0,440047,0.007136,19,4
...,...,...,...,...,...,...,...
357026,0.851124,16,0,442847,0.003803,17,1
296740,0.851124,32,0,442894,0.011180,8,3
418791,0.851124,69,0,440079,0.037986,8,5


In [278]:
train_matrix = csr_matrix(train)

In [279]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(train_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [281]:
import joblib

In [283]:
joblib.dump(model_knn, 'model_knn_book')

['model_knn_book']

In [177]:
id_user = test[test.index.get_level_values('ИД читателя') == 4992]

In [178]:
id_sim = []
for i in range(len(id_user)):
    #Определяем дистанцию между авторами
    distances, indices = model_knn.kneighbors(id_user.iloc[i].values.reshape(1, -1), n_neighbors = 20)
    for j in range(0, len(indices.flatten())):
        id_sim.append(indices.flatten()[j])
counter=collections.Counter(id_sim)
id_sim = sorted(counter, key=counter.get, reverse=True)[:20]

In [84]:
def freqInfo(df, col):
    list_df = []
    for c in df:
        list_df.append(train[train.index.get_level_values('ИД читателя') ==
                               train.index[c]][col].drop_duplicates().values.tolist())
    recom_list = []
    for i in range(len(list_df)):
        for j in range(len(list_df[i])):
            recom_list.append(list_df[i][j])
    counter=collections.Counter(recom_list)
    return (sorted(counter, key=counter.get, reverse=True)[:20])

In [179]:
freqInfo(id_sim, 'ИД Каталожной записи')

[3609,
 4381,
 4269,
 4721,
 10477,
 416634,
 3987,
 29521,
 452976,
 434382,
 454719,
 423023,
 433269,
 420426,
 433592,
 439138,
 454695,
 7929,
 4232,
 4584]

In [182]:
result_i = pd.DataFrame([])
for c in freqInfo(id_sim, 'ИД Каталожной записи'):
    result_i = result_i.append(book_user[book_user['ИД Каталожной записи'] == c])

In [184]:
result_i = pd.merge(result_i['ИД Каталожной записи'],catalog,how='left',left_on='ИД Каталожной записи',right_on='doc_id').drop(columns=['doc_id','genre'])

In [186]:
result_i.drop_duplicates()

,ИД Каталожной записи,p100a,p245a,p260a,p260b,p260c,p490a,p650a,p521a
0,3609,Янссон Туве,Сказки Долины муми-троллей,Санкт-Петербург,Азбука,2012,Муми-тролль и все-все-все,Художественная литература,NaN
44,4381,Минаев Сергей Сергеевич,Духless 21 века. Селфи,Москва,АСТ ; Жанры,2015,NaN,Художественная литература,18+
79,4269,Бут Стивен,Танцуя с девственницами,Санкт-Петербург,Азбука-классика,2008,NaN,Художественная литература,NaN
90,4721,ДиКамилло Кейт,Райми Найтингейл - девочка с лампой,Москва,Махаон ; Азбука-Аттикус,2017,NaN,Художественная литература ; Литература для дет...,6+
312,10477,Вамба,Дневник Джанни Урагани,Москва,Самокат,2015,NaN,Художественная литература,6+
415,416634,Вебб Холли,Призрак кошки,Москва,Эксмо,2018,Мейзи Хитчинс. Приключения девочки-детектива,Художественная литература,12+
446,3987,Акунин Борис,Черный город,Москва,Захаров,2013,Новый детективъ,Художественная литература,16+
654,29521,Коэльо Пауло,Победитель остается один,Москва,АСТ,2012,NaN,Художественная литература,NaN
669,452976,Кобах Сергей Львович,Злобная кухня,Москва,Эксмо,2011,NaN,Технология общественного питания. Кулинария,NaN
673,434382,Юри Уильям,"Преодолевая НЕТ, или Переговоры с трудными л...",Москва,Эксмо,2008,NaN,Психология развития и возрастная психология,NaN


Переход в мероприятия

In [110]:
freqInfo(id_sim, 'classif')

[20, 14, 8, 13, 17, 16, 24, 11, 10, 12, 7, 19, 1]

In [111]:
freqInfo(id_sim, 'age_range_code')

[4, 3, 5, 2]

In [71]:
oraniz_test['возраст'] = oraniz_test['возраст'].astype(int)

In [73]:
oraniz_test = age_range(oraniz_test,'возраст')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [74]:
oraniz_test = oraniz_test.sort_values(by=['age_range'])

In [87]:
oraniz_test['age_range_code'] = oraniz_test.age_range.astype('category').cat.codes

Переход

In [116]:
result = pd.DataFrame([])
for c in freqInfo(id_sim, 'age_range_code'):
    for cl in freqInfo(id_sim, 'classif'):
        result = result.append(oraniz_test[(oraniz_test.age_range_code == c) & (oraniz_test.classif == cl)])

In [121]:
result = result[['Наименование_услуги', 'id_организации']].drop_duplicates()

In [126]:
result = pd.merge(result, data3, how='left', on='id_организации')

Рекомендация по истории пользователя Библиотек по посещению кружков

In [127]:
result

,Наименование_услуги,id_организации,полное_наименование,краткое_наименование,улица,Метро
0,Хор русской песни На завалинке,10912,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ТКС ""Орехово""",Борисовский проезд,Домодедовская
1,Хор русской песни На завалинке,10912,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ТКС ""Орехово""",Борисовский проезд,Красногвардейская
2,Хор русской песни На завалинке,10912,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ТКС ""Орехово""",Борисовский проезд,Шипиловская
3,Хор русской песни На завалинке,10912,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ТКС ""Орехово""",Борисовский проезд,Орехово
4,Хоровая мастерская,21810,Государственное бюджетное учреждение культуры...,"ГБУК г. Москвы ""ММКЦ""",Ярославское шоссе,ВДНХ
...,...,...,...,...,...,...
4029,Балалайка комплексная 7-летняя программа,48,Государственное бюджетное учреждение дополните...,ГБОУ ДОД г.Москвы «Детская школа искусств имен...,Палехская улица,ВДНХ
4030,Балалайка комплексная 5-летняя программа,48,Государственное бюджетное учреждение дополните...,ГБОУ ДОД г.Москвы «Детская школа искусств имен...,Палехская улица,Бабушкинская
4031,Балалайка комплексная 5-летняя программа,48,Государственное бюджетное учреждение дополните...,ГБОУ ДОД г.Москвы «Детская школа искусств имен...,Палехская улица,Ботанический сад
4032,Балалайка комплексная 5-летняя программа,48,Государственное бюджетное учреждение дополните...,ГБОУ ДОД г.Москвы «Детская школа искусств имен...,Палехская улица,ВДНХ


Рекомендательный алгоритм по кружкам

In [284]:
train, test= train_test_split(oraniz_test.drop(columns=['Наименование_услуги','age_range']).set_index(['id_ученика']).astype(int),stratify=oraniz_test['classif'],
                              random_state = 256,test_size=0.2)

In [285]:
train_matrix = csr_matrix(train)

In [286]:
model_knn_org = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn_org.fit(train_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [287]:
joblib.dump(model_knn_org, 'model_knn_org')

['model_knn_org']

In [144]:
id_user = test[test.index.get_level_values('id_ученика') == 3085269]

In [145]:
id_sim = []
for i in range(len(id_user)):
    #Определяем дистанцию между авторами
    distances, indices = model_knn_org.kneighbors(id_user.iloc[i].values.reshape(1, -1), n_neighbors = 20)
    for j in range(0, len(indices.flatten())):
        id_sim.append(indices.flatten()[j])
counter=collections.Counter(id_sim)
id_sim = sorted(counter, key=counter.get, reverse=True)[:20]

In [149]:
def freqInfoT(df, col):
    list_df = []
    for c in df:
        list_df.append(train[train.index.get_level_values('id_ученика') ==
                               train.index[c]][col].drop_duplicates().values.tolist())
    recom_list = []
    for i in range(len(list_df)):
        for j in range(len(list_df[i])):
            recom_list.append(list_df[i][j])
    counter=collections.Counter(recom_list)
    return (sorted(counter, key=counter.get, reverse=True)[:20])

In [150]:
freqInfoT(id_sim, 'id_организации')

[10887, 10920, 10860, 10912, 10917, 10873, 10850, 10854, 10843, 10858]

Рекомендуем другие кружки

In [167]:
result_i = pd.DataFrame([])
for c in freqInfoT(id_sim, 'id_организации'):
    result_i = result_i.append(oraniz_test[oraniz_test['id_организации'] == c])

In [170]:
result_i = result_i[['Наименование_услуги','id_организации','years', 'weeks', 'months', 'days']]

In [172]:
result_i = pd.merge(result_i, data3, how='left', on='id_организации')

Результат

In [173]:
result_i

,Наименование_услуги,id_организации,years,weeks,months,days,полное_наименование,краткое_наименование,улица,Метро
0,"Секция «ХУДОЖЕСТВЕННАЯ ГИМНАСТИКА», гр №5",10887,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ЦКИ ""Меридиан""",Профсоюзная улица,Калужская
1,"Секция единоборств ШАНГИ, гр №1",10887,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ЦКИ ""Меридиан""",Профсоюзная улица,Калужская
2,"Секция единоборств ШАНГИ, гр №1",10887,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ЦКИ ""Меридиан""",Профсоюзная улица,Калужская
3,"Секция единоборств ШАНГИ, гр №1",10887,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ЦКИ ""Меридиан""",Профсоюзная улица,Калужская
4,"Секция единоборств ШАНГИ, гр №1",10887,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ЦКИ ""Меридиан""",Профсоюзная улица,Калужская
...,...,...,...,...,...,...,...,...,...,...
113687,"Оперетта на Таганке, бесплатная основа",10858,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ДК ""Стимул""",Сибирский проезд,NaN
113688,"Оперетта на Таганке, бесплатная основа",10858,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ДК ""Стимул""",Сибирский проезд,NaN
113689,"Оперетта на Таганке, бесплатная основа",10858,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ДК ""Стимул""",Сибирский проезд,NaN
113690,"Оперетта на Таганке, бесплатная основа",10858,5,0,0,0,Государственное бюджетное учреждение культуры ...,"ГБУК г. Москвы ""ДК ""Стимул""",Сибирский проезд,NaN


Переход в книги

In [151]:
freqInfoT(id_sim, 'classif')

[19, 20, 17, 16, 13, 24]

In [152]:
freqInfoT(id_sim, 'age_range_code')

[2]

In [155]:
result = pd.DataFrame([])
for c in freqInfoT(id_sim, 'age_range_code'):
    for cl in freqInfoT(id_sim, 'classif'):
        result = result.append(book_user[(book_user.age_range_code == c) & (book_user.classif == cl)])

In [163]:
result = pd.merge(result['ИД Каталожной записи'],catalog,how='left',left_on='ИД Каталожной записи',right_on='doc_id').drop(columns=['doc_id','genre'])

Результат работы

In [164]:
result

,ИД Каталожной записи,p100a,p245a,p260a,p260b,p260c,p490a,p650a,p521a
0,406218,Хокинг Джейн,Быть Хокингом,Москва,Эксмо,2018,Civiliзация,Физика,12+
1,440047,NaN,Советский спорт,NaN,NaN,2019,NaN,Физическая культура и спорт. Туризм,NaN
2,10673,NaN,Вокруг света,NaN,NaN,2018,NaN,Физическая география,12+
3,510630,Тарасов Владимир Иванович,"Деньги, кредит, банки",Минск,Мисанта,2003,NaN,Финансы. Банковское дело. Рынок ценных бумаг. ...,NaN
4,9320,Маклин Алистер,Капитан Кук,Москва,Центрполиграф,2001,Великие географические открытия,Физическая география,NaN
...,...,...,...,...,...,...,...,...,...
11447,412995,NaN,Экология города,Ростов-на-Дону,МарТ,2008,Учебный курс,Экология,NaN
11448,78,NaN,Охрана окружающей среды,Москва,Академия,2014,Высшее образование. Бакалавриат,Экология ; Охрана окружающей среды в целом. Ра...,NaN
11449,439076,NaN,Этикет от А до Я,Москва,АСТ,1996,NaN,Этика. Культура поведения и этикет,NaN
11450,82,Баландин Рудольф Константинович,Экология. Человек и природа,Москва,ОЛМА-Пресс,2001,NaN,Экология,NaN


Переход в мероприятия

In [214]:
def Average(lst): 
    return sum(lst) / len(lst)

In [257]:
result_e = event[(event['from'] < Average(freqInfo(id_sim, 'Возраст')[:5]) ) &
                 (event['to'] > Average(freqInfo(id_sim, 'Возраст')[:5]) )].copy()

In [221]:
result_e

,id,Статус,Направленность мероприятия,Дата начала мероприятия,Дата окончания мероприятия,Место проведения,Доступность мероприятия для лиц с ОВЗ,Возрастная категория,Код категории мероприятия,from,to,latitude,longitude
0,1273,Утверждено,культурно-досуговое,2020-01-01 00:00:00,2020-12-31 00:00:00,"Россия, Москва, Москва, ул. Профсоюзная, д.123А",недоступно,от 0 до 90,4,0,90.0,37.539060,55.623961
1,1278,Утверждено,культурно-досуговое,2020-01-01 00:00:00,2020-12-31 00:00:00,"Россия, Москва, Москва, ст. м. Проспект Вернад...",недоступно,от 0 до 90,4,0,90.0,37.505831,55.676910
2,1279,Утверждено,культурно-досуговое,2020-01-01 00:00:00,2020-12-31 00:00:00,"Россия, Москва, Москва, ст. м. Ясенево",недоступно,от 0 до 90,4,0,90.0,37.533400,55.606182
3,1280,Утверждено,культурно-досуговое,2020-01-01 00:00:00,2020-12-31 00:00:00,"Россия, Москва, Москва, ст. м. Тушино",недоступно,от 0 до 90,4,0,90.0,37.440263,55.827338
4,1282,Утверждено,культурно-досуговое,2020-01-01 00:00:00,2020-12-31 00:00:00,"Россия, Москва, Москва, Загородное шоссе, вл. 2",недоступно,от 0 до 90,4,0,90.0,37.605400,55.698457
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3729,583,Запланировано,просветительское,2020-12-15 00:00:00,2020-12-15 00:00:00,"Очаковское шоссе 21, корпус 2",недоступно,от 16 до 70,8,16,70.0,37.451815,55.690583
3730,584,Запланировано,просветительское,2020-12-15 00:00:00,2020-12-15 00:00:00,"Очаковское шоссе 21, корпус 2",недоступно,от 16 до 70,8,16,70.0,37.451815,55.690583
3731,1033,Запланировано,культурно-досуговое,2020-12-15 00:00:00,2020-12-15 00:00:00,Шушенская ул. 7,недоступно,от 12 до 90,4,12,90.0,37.683230,55.869723
3732,1449,Запланировано,культурно-досуговое,2020-12-15 00:00:00,2020-12-15 00:00:00,"Крылатская ул. 29, корпус 2",недоступно,от 0 до 99,4,0,99.0,37.410879,55.770056


In [258]:
from sklearn.cluster import KMeans ,AgglomerativeClustering

#Создает 125  кластеров, используя average кластеризацию. (Районов в Москве)
agc = AgglomerativeClustering(n_clusters =125 , affinity='cosine', linkage='average')
result_e['pickup cluster'] = agc.fit_predict(result_e[['longitude','latitude']])

#Создает 125  кластеров, используя алгоритм кластеризации k-средних.
kmeans = KMeans(125)
clusters = kmeans.fit_predict(result_e[['longitude','latitude']])
result_e['pickup cluster'] = kmeans.predict(result_e[['longitude','latitude']])

Проверка на геопозицию

In [259]:
position = "Красная площадь, 3"

In [260]:
longitude = float(client.coordinates(position + "Москва")[1])
latitude = float(client.coordinates(position + "Москва")[0])
print(longitude,latitude)

55.754638 37.621633


In [261]:
def haversine_distance(row):
    lat_p, lon_p = row['longitude'], row['latitude']
    lat_d, lon_d = longitude,latitude
    radius =  6371 # km

    dlat = np.radians(lat_d - lat_p)
    dlon = np.radians(lon_d - lon_p)
    a = np.sin(dlat/2) * np.sin(dlat/2) + np.cos(np.radians(lat_p)) * np.cos(np.radians(lat_d)) * np.sin(dlon/2) * np.sin(dlon/2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = radius * c

    return distance

In [262]:
result_e['distance'] = result_e.apply(haversine_distance, axis = 1)

In [256]:
result_e = result_e.sort_values(by='distance', ascending=True)[:50]

In [246]:
result_e = result_e.sort_values(by='Дата начала мероприятия', ascending=True)[:25]

In [249]:
result_e[result_e['Доступность мероприятия для лиц с ОВЗ'] == 'доступно'].drop(columns=['id', 'Статус', 'Код категории мероприятия','from','to','latitude',	'longitude', 'pickup cluster', 'distance'])

,Направленность мероприятия,Дата начала мероприятия,Дата окончания мероприятия,Место проведения,Доступность мероприятия для лиц с ОВЗ,Возрастная категория
20,культурно-досуговое,2020-01-09 00:00:00,2020-12-23 00:00:00,"Бобров пер. 6, строение 2, Тургеневская гостин...",доступно,от 12 до 99
22,культурно-досуговое,2020-01-09 00:00:00,2020-12-23 00:00:00,"Бобров пер. 6, строение 2, Тургеневская гостин...",доступно,от 12 до 99
21,культурно-досуговое,2020-01-09 00:00:00,2020-12-23 00:00:00,"Бобров пер. 6, строение 2, Тургеневская гостин...",доступно,от 12 до 99
51,культурно-досуговое,2020-03-02 00:00:00,2020-12-30 00:00:00,"Бобров пер. 6, строение 1, Большой читальный з...",доступно,от 16 до 99
52,культурно-досуговое,2020-03-02 00:00:00,2020-12-30 00:00:00,"Бобров пер. 6, строение 1, Большой читальный з...",доступно,от 16 до 99
50,культурно-досуговое,2020-03-02 00:00:00,2020-12-30 00:00:00,"Бобров пер. 6, строение 1, Большой читальный з...",доступно,от 16 до 99
891,культурно-досуговое,2020-10-21 00:00:00,2020-10-21 00:00:00,Никитский бульв. 7А,доступно,от 12 до 70
1045,культурно-досуговое,2020-10-22 00:00:00,2020-10-22 00:00:00,"Страстной бульв. 6, строение 2",доступно,от 16 до 70
954,культурно-досуговое,2020-10-22 00:00:00,2020-10-22 00:00:00,"Чистопрудный бульв. 23, строение 1",доступно,от 18 до 75
1290,культурно-досуговое,2020-10-26 00:00:00,2020-10-26 00:00:00,"Чистопрудный бульв. 23, строение 1",доступно,от 18 до 75


Категории

In [201]:
pd.merge(book_user[['ИД Каталожной записи','genre','classif']],catalog,how='left',left_on='ИД Каталожной записи',right_on='doc_id')[['p650a','classif']].drop_duplicates().to_excel('table_book.xlsx', index = False)

In [202]:
oraniz_test[['Наименование_услуги','classif']].drop_duplicates().to_excel('table_org.xlsx', index = False)